In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio, display
import librosa
import librosa.display

In [ ]:
# --- Step 1: Load the WAV files ---
file1 = 'dataset\Malauhog\malauhog_20251126-233235.wav'
file2 = 'dataset\Malauhog\malauhog_20251126-233438.wav'

Fs1, y1 = wavfile.read(file1)
Fs2, y2 = wavfile.read(file2)

In [ ]:
# --- Step 2: Convert to mono if stereo ---
if y1.ndim > 1:
    y1 = y1.mean(axis=1)
if y2.ndim > 1:
    y2 = y2.mean(axis=1)

In [ ]:
# --- Step 3: FFT calculation ---
def compute_fft(y, Fs):
    N = len(y)
    t = np.arange(N)/Fs
    Y = np.fft.fft(y)
    f = np.fft.fftfreq(N, d=1/Fs)
    P2 = np.abs(Y/N)
    P1 = P2[:N//2]*2
    f1 = f[:N//2]
    return t, y, f1, P1

t1, y1, f1, P1 = compute_fft(y1, Fs1)
t2, y2, f2, P2 = compute_fft(y2, Fs2)

In [ ]:
# --- Step 4: Plot time-domain signals ---
plt.figure(figsize=(14,8))
plt.subplot(2,2,1)
plt.plot(t1, y1)
plt.title('Time Domain - File 1')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.grid(True)

plt.subplot(2,2,2)
plt.plot(t2, y2)
plt.title('Time Domain - File 2')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.grid(True)

In [ ]:
# --- Step 5: Plot frequency spectra ---
plt.figure(figsize=(12,5))
plt.plot(f1, P1)
plt.title('Frequency Spectrum - File 1')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude')
plt.grid(True)
plt.show()

plt.figure(figsize=(12,5))
plt.plot(f2, P2)
plt.title('Frequency Spectrum - File 2')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude')
plt.grid(True)
plt.show()

In [ ]:
# --- Step 6: Play audio files ---
print("Playing File 1:")
display(Audio(y1, rate=Fs1))
print("Playing File 2:")
display(Audio(y2, rate=Fs2))

In [ ]:
# --- Step 7: Frequency characteristics ---
def frequency_characteristics(frequencies, magnitude):
    dominant_freq = frequencies[np.argmax(magnitude)]
    mean_freq = np.sum(frequencies * magnitude) / np.sum(magnitude)
    bandwidth = frequencies[np.where(magnitude > (0.5 * np.max(magnitude)))][[-1]][0] - frequencies[np.where(magnitude > (0.5 * np.max(magnitude)))][0]
    return dominant_freq, mean_freq, bandwidth

dom1, mean1, bw1 = frequency_characteristics(f1, P1)
dom2, mean2, bw2 = frequency_characteristics(f2, P2)

print(f"\nFrequency characteristics of File 1:")
print(f"Dominant Frequency: {dom1:.2f} Hz")
print(f"Mean Frequency: {mean1:.2f} Hz")
print(f"Approximate Bandwidth: {bw1:.2f} Hz")

print(f"\nFrequency characteristics of File 2:")
print(f"Dominant Frequency: {dom2:.2f} Hz")
print(f"Mean Frequency: {mean2:.2f} Hz")
print(f"Approximate Bandwidth: {bw2:.2f} Hz")

In [ ]:
# --- Step 8: Spectral tilt and shape ---
def spectral_features(y, Fs):
    y_float = y.astype(float)
    S = np.abs(librosa.stft(y_float))
    centroid = librosa.feature.spectral_centroid(S=S, sr=Fs)
    bandwidth = librosa.feature.spectral_bandwidth(S=S, sr=Fs)
    rolloff = librosa.feature.spectral_rolloff(S=S, sr=Fs)
    flatness = librosa.feature.spectral_flatness(S=S)

    # Spectrum tilt (slope of log spectrum)
    freqs = librosa.fft_frequencies(sr=Fs)
    log_spectrum = np.log(np.mean(S, axis=1) + 1e-8)
    slope, intercept = np.polyfit(freqs, log_spectrum, 1)

    return centroid, bandwidth, rolloff, flatness, slope, S, freqs, log_spectrum

cent1, bw_spec1, roll1, flat1, tilt1, S1, freqs1, logspec1 = spectral_features(y1, Fs1)
cent2, bw_spec2, roll2, flat2, tilt2, S2, freqs2, logspec2 = spectral_features(y2, Fs2)

In [ ]:
# --- Step 9: Plot spectral shape and tilt ---
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(freqs1, logspec1, label='File 1', color='b')
plt.plot(freqs1, np.polyval([tilt1, np.mean(logspec1)], freqs1), 'k--', label='Tilt Line')
plt.title('Spectral Tilt - File 1')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Log Magnitude')
plt.grid(True)
plt.legend()

plt.subplot(1,2,2)
plt.plot(freqs2, logspec2, label='File 2', color='r')
plt.plot(freqs2, np.polyval([tilt2, np.mean(logspec2)], freqs2), 'k--', label='Tilt Line')
plt.title('Spectral Tilt - File 2')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Log Magnitude')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

print("\nSpectral Shape and Tilt (File 1):")
print(f"Spectral Centroid: {np.mean(cent1):.2f} Hz")
print(f"Spectral Bandwidth: {np.mean(bw_spec1):.2f} Hz")
print(f"Spectral Rolloff: {np.mean(roll1):.2f} Hz")
print(f"Spectral Flatness: {np.mean(flat1):.4f}")
print(f"Spectral Tilt (slope): {tilt1:.6f}")

print("\nSpectral Shape and Tilt (File 2):")
print(f"Spectral Centroid: {np.mean(cent2):.2f} Hz")
print(f"Spectral Bandwidth: {np.mean(bw_spec2):.2f} Hz")
print(f"Spectral Rolloff: {np.mean(roll2):.2f} Hz")
print(f"Spectral Flatness: {np.mean(flat2):.4f}")
print(f"Spectral Tilt (slope): {tilt2:.6f}")

In [ ]:
# --- Step 10: MFCC feature extraction ---
def compute_mfcc(y, Fs):
    y_float = y.astype(float)
    mfccs = librosa.feature.mfcc(y=y_float, sr=Fs, n_mfcc=13)
    return mfccs

mfcc1 = compute_mfcc(y1, Fs1)
mfcc2 = compute_mfcc(y2, Fs2)

In [ ]:
# --- Step 11: Plot MFCCs ---
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
librosa.display.specshow(mfcc1, sr=Fs1, x_axis='time')
plt.colorbar()
plt.title('MFCCs - File 1')

plt.subplot(1,2,2)
librosa.display.specshow(mfcc2, sr=Fs2, x_axis='time')
plt.colorbar()
plt.title('MFCCs - File 2')

plt.tight_layout()
plt.show()

In [ ]:
# --- Step 12: Print MFCC summary ---
print("\nMFCC feature summary for File 1 (mean of each coefficient):")
print(np.round(np.mean(mfcc1, axis=1), 2))

print("\nMFCC feature summary for File 2 (mean of each coefficient):")
print(np.round(np.mean(mfcc2, axis=1), 2))